# Task1
Build a user retention curve (Rolling Retention). What percentage returned on any day from N to 30, where N = {0, 1, 2 ... 14}.

```
with logins as (
    select 
        id_ru, date_login
    from `logins` 
    where true
        and (date_login between '2018-09-15' and '2018-12-15')
),

reg as (
    select 
        id_ru, date_registration
    from `regular_users` 
    where true
        and (date_registration between '2018-09-15' and '2018-11-01')
),

select_days as (
    select 
        id_ru,
        date_diff(l.date_login, ru.date_registration, day) as num_day,
    from reg
        left join logins lo using(id_ru)
    where true
        and date_diff(lo.date_login, ru.date_registration, day) <= 30
),

last_log as (
    select 
        id_ru,  
        max(num_day) as last_retention
    from select_days
    group by 1
),

group_by_last as (
    select 
        last_retention,
        count(id_ru) as users
    from last_log
    group by 1
    order by 1
)

select 
    last_retention,
    sum(users) over(order by last_retention desc) / (select count(id_ru) from ru) as num_day_rolling
from group_by_last
order by 1
```